In [1]:
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import pandas as pd

# Set the width and height for loading images
w, h = 250, 250  # You can adjust these values based on your requirements


# List of folders with their respective label1 and label2 values
folder_data = [
    {'folder': 'C:\\Users\\Asus\\OneDrive\\Desktop\\6th Sem Project\\archive\\Comprehensive Disaster Dataset(CDD)\\Fire_Disaster\\Wild_Fire', 'label1': 'Fire Disaster', 'label2': 'Wild Fire'},
    {'folder': 'C:\\Users\\Asus\\OneDrive\\Desktop\\6th Sem Project\\archive\\Comprehensive Disaster Dataset(CDD)\\Land_Disaster\\Drought', 'label1': 'Land Disaster', 'label2': 'Drought'},
    {'folder': 'C:\\Users\\Asus\\OneDrive\\Desktop\\6th Sem Project\\archive\\Comprehensive Disaster Dataset(CDD)\\Non_Damage\\human', 'label1': 'Human', 'label2': 'Human'},
    {'folder': 'C:\\Users\\Asus\\OneDrive\\Desktop\\6th Sem Project\\archive\\Comprehensive Disaster Dataset(CDD)\\Non_Damage\\Non_Damage_Wildlife_Forest', 'label1': 'Nature', 'label2': 'Undamaged Forest'},
    {'folder': "C:\\Users\\Asus\\OneDrive\\Desktop\\6th Sem Project\\archive\\Sea", 'label1': 'Sea', 'label2': 'Sea'},
    {'folder': "C:\\Users\\Asus\\OneDrive\\Desktop\\6th Sem Project\\archive\\Comprehensive Disaster Dataset(CDD)\\Non_Damage\\Rain", 'label1': 'Rain', 'label2': 'Water Disaster'},
    {'folder': 'C:\\Users\\Asus\\OneDrive\\Desktop\\6th Sem Project\\archive\\Comprehensive Disaster Dataset(CDD)\\Water_Disaster', 'label1': 'Water Disaster', 'label2': 'Water Disaster'}
    # Add the remaining folders and label values
]

# Combine file paths and label values into a list of dictionaries
data_list = []

for folder_info in folder_data:
    folder_path = folder_info['folder']
    
    if os.path.exists(folder_path):
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    image_path = os.path.join(root, file)
                    data_list.append({'image_path': image_path, 'label1': folder_info['label1'], 'label2': folder_info['label2']})
    else:
        print(f"Folder not found: {folder_info['folder']}")

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_list)

# Split data into training and testing sets with stratification
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label1'])

# Load images and labels into arrays
def load_images_and_labels(data):
    images = []
    labels1 = []
    labels2 = []
    for index, row in data.iterrows():
        try:
            img = load_img(row['image_path'], target_size=(w, h))
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
            labels1.append(row['label1'])
            labels2.append(row['label2'])
        except Exception as e:
            print(f"Error loading image {row['image_path']}: {e}")

    return np.array(images), np.array(labels1), np.array(labels2)

# Load images and labels for training and testing sets
train_images, train_labels1, train_labels2 = load_images_and_labels(train_data)
test_images, test_labels1, test_labels2 = load_images_and_labels(test_data)


In [2]:
import pandas as pd

# Assuming df is your DataFrame
# df['label'] is the column for which you want to get the value counts

value_counts = df['label1'].value_counts()
print(value_counts)


label1
Nature            4900
Water Disaster     602
Sea                556
Fire Disaster      361
Human              297
Rain               207
Land Disaster      191
Name: count, dtype: int64


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.datasets import mnist
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from PIL import Image
import os

In [4]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Determine the number of unique classes for label1 and label2
num_classes1 = len(df['label1'].unique())
num_classes2 = len(df['label2'].unique())

# Create a LabelEncoder for each category
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()

# Fit the LabelEncoder on the combined set of training and test labels
combined_labels1 = np.concatenate([train_labels1, test_labels1])
combined_labels2 = np.concatenate([train_labels2, test_labels2])

label_encoder1.fit(combined_labels1)
label_encoder2.fit(combined_labels2)

# Transform string labels to numerical indices
train_labels1_indices = label_encoder1.transform(train_labels1)
train_labels2_indices = label_encoder2.transform(train_labels2)

test_labels1_indices = label_encoder1.transform(test_labels1)
test_labels2_indices = label_encoder2.transform(test_labels2)

# Apply to_categorical on numerical indices
train_labels1_one_hot = to_categorical(train_labels1_indices, num_classes=num_classes1)
train_labels2_one_hot = to_categorical(train_labels2_indices, num_classes=num_classes2)

test_labels1_one_hot = to_categorical(test_labels1_indices, num_classes=num_classes1)
test_labels2_one_hot = to_categorical(test_labels2_indices, num_classes=num_classes2)


In [5]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate

# Define input shape
input_shape = (250, 250, 3)  # Adjust based on your image size and channels

# Define the input layer
input_layer = Input(shape=input_shape, name='input_layer')

# Convolutional block
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)
flatten = Flatten()(pool2)

# First output branch for label1
dense1 = Dense(128, activation='relu')(flatten)
output1 = Dense(7, activation='softmax', name='output1')(dense1)

# Combine both output branches
model = Model(inputs=input_layer, outputs=[output1])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
from keras.models import Model

# Create model1 with output1 branch
model1 = Model(inputs=model.input, outputs=model.get_layer('output1').output)
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
# Define callbacks
es1 = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=2, verbose=1, restore_best_weights=True)
mc1 = ModelCheckpoint("Finally_Final_DDS.keras", monitor="val_accuracy", verbose=1, save_best_only=True)


In [8]:
# Fit method for training model1
#model1.fit(
 #   x=train_images,
  #  y=train_labels1_one_hot,
   # epochs=10,
    #validation_data=(test_images, test_labels1_one_hot)
#)

In [9]:
# Fit method for training model2
model1.fit(
    x=train_images,
    y=train_labels1_one_hot,
    epochs=3,
    validation_data=(test_images, test_labels1_one_hot),
    callbacks=[es1, mc1]    
)   

Epoch 1/3


178/178 [==============================] - ETA: 0s - loss: 0.7997 - accuracy: 0.8183
Epoch 1: val_accuracy improved from -inf to 0.88616, saving model to Finally_Final_DDS.keras
178/178 [==============================] - 107s 583ms/step - loss: 0.7997 - accuracy: 0.8183 - val_loss: 0.3268 - val_accuracy: 0.8862
Epoch 2/3
178/178 [==============================] - ETA: 0s - loss: 0.2041 - accuracy: 0.9320
Epoch 2: val_accuracy improved from 0.88616 to 0.92621, saving model to Finally_Final_DDS.keras
178/178 [==============================] - 98s 551ms/step - loss: 0.2041 - accuracy: 0.9320 - val_loss: 0.2347 - val_accuracy: 0.9262
Epoch 3/3
178/178 [==============================] - ETA: 0s - loss: 0.0885 - accuracy: 0.9759
Epoch 3: val_accuracy did not improve from 0.92621
178/178 [==============================] - 97s 544ms/step - loss: 0.0885 - accuracy: 0.9759 - val_loss: 0.2749 - val_accuracy: 0.9178


In [10]:
# Evaluate model1
loss1, accuracy1 = model1.evaluate(test_images, test_labels1_one_hot, verbose=1)
print(f"Model1 Accuracy: {accuracy1 * 100:.2f}%")


45/45 [==============================] - 4s 90ms/step - loss: 0.2749 - accuracy: 0.9178
Model1 Accuracy: 91.78%
